# Example: Compound

In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(project_root, "src")

if src_path not in sys.path:
    sys.path.insert(0, src_path)

print("PYTHONPATH:", sys.path[0])

PYTHONPATH: c:\Users\are10\Documents\BAFSTU\code\pyBioDatafusemain\pyBiodatafuse\src


In [2]:
# Import modules
import pandas as pd

from pyBiodatafuse import id_mapper
from pyBiodatafuse.annotators import intact
from pyBiodatafuse.graph import generator
from pyBiodatafuse.utils import combine_sources

# Load list of metabolites/chemicals/compounds

In [3]:
metabolites_of_interest = """15361
16393"""

metabolite_list = metabolites_of_interest.split("\n")
len(metabolite_list)

2

In [4]:
data_input = pd.DataFrame(metabolite_list, columns=["identifier"])
data_input.head()

,identifier
0,15361
1,16393


### Entity resolution using BridgeDB

In [5]:
bridgedb_compound_df, bridgdb_metadata = id_mapper.bridgedb_xref(
    identifiers=data_input,
    input_species="Human",
    input_datasource="ChEBI",
    output_datasource="All",
)
bridgedb_compound_df.head(10)

,identifier,identifier.source,target,target.source
0,15361,ChEBI,Q27089397,Wikidata
1,15361,ChEBI,96901,ChemSpider
2,15361,ChEBI,CHEBI:15361,ChEBI
3,15361,ChEBI,107735,PubChem Compound
4,15361,ChEBI,LCTONWCANYUPML-UHFFFAOYSA-M,InChIKey
5,15361,ChEBI,15361,ChEBI
6,15361,ChEBI,57-60-3,CAS
7,16393,ChEBI,123-78-4,CAS
8,16393,ChEBI,5280335,PubChem Compound
9,16393,ChEBI,Q46298,Wikidata


### Gene Interactions from IntAct

In [6]:
intact_compound_df, intact_compound_metadata = intact.get_compound_interactions(
    bridgedb_compound_df, interaction_type="both"
)
intact_compound_df.head(10)

,identifier,identifier.source,target,target.source,IntAct_compound_interactions
0,15361,ChEBI,CHEBI:15361,ChEBI,"[{'interaction_id': 'EBI-9301798', 'interactor..."
1,16393,ChEBI,CHEBI:16393,ChEBI,"[{'interaction_id': 'EBI-26355511', 'interacto..."


In [7]:
intact_compound_df["IntAct_compound_interactions"][0]

[{'interaction_id': 'EBI-9301798',
  'interactor_id_A': 'EBI-9096',
  'interactor_id_B': 'EBI-6621808',
  'binary_interaction_id': 13894862,
  'confidence_values': ['intact-miscore:0.44'],
  'score': 0.44,
  'biological_role_A': 'enzyme',
  'biological_role_B': 'enzyme target',
  'type': 'enzymatic reaction',
  'stoichiometry_A': '0-0',
  'stoichiometry_B': '0-0',
  'detection_method': 'enzymatic study',
  'detection_method_id': 'MI:0415',
  'host_organism': 'In vitro',
  'interactor_A_name': 'ilvb_yeast',
  'interactor_B_name': 'pyruvate',
  'interactor_A_species': 'Saccharomyces cerevisiae',
  'interactor_B_species': 'Chemical synthesis (Chemical synthesis)',
  'molecule_A': 'ILV2',
  'molecule_B': 'pyruvate',
  'id_A': 'P07342',
  'id_B': 'CHEBI:15361',
  'pubmed_publication_id': '16390333',
  'altIdsA': ['P07342 (uniprotkb)', 'D6VZT1 (uniprotkb)', 'EBI-9096 (intact)'],
  'altIdsB': ['CHEBI:15361 (chebi)', 'EBI-6621808 (intact)'],
  'intact_link_to': None},
 {'interaction_id': 'EBI-

In [8]:
combined_df = combine_sources(
    bridgedb_compound_df,
    [intact_compound_df],
)

In [9]:
pygraph = generator.build_networkx_graph(combined_df)

Building graph: 100%|██████████| 2/2 [00:00<00:00, 969.56it/s]


In [10]:
combined_df.shape

(2, 5)

In [11]:
from pyBiodatafuse.graph import cytoscape

cytoscape.load_graph(pygraph, network_name="Test network")

Applying default style...
Applying preferred layout
